### Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

%matplotlib inline
sys.path.append("..")

from utils import label_map_util
from utils import visualization_utils as vis_util
from utils import path_detection_utils as path_util
from utils import data_collection_util as data_util

### Loading in model 

In [2]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

### Object Detection on specific frame

In [3]:
path1 = 'video_frames/frame199.jpg'
path2 = 'video_frames/frame200.jpg'
path3 = 'video_frames/frame201.jpg'

path4 = 'test_images/pic.jpg'

image1 = Image.open(path1)
width1, height1 = image1.size
image_np1 = path_util.load_image_into_numpy_array(image1)
boxes1, scores1, classes1 = path_util.get_segmentation(path1,detection_graph, category_index, False)
human_boxes1, human_scores1 = path_util.remove_non_human(boxes1, scores1, classes1)
good_boxes1, good_scores1 = path_util.remove_low_prob(human_boxes1, human_scores1)
good_boxes1, good_scores1 = path_util.remove_poorly_sized_people(good_boxes1, good_scores1)

image2 = Image.open(path2)
width2, height2 = image2.size
image_np2 = path_util.load_image_into_numpy_array(image2)
boxes2, scores2, classes2 = path_util.get_segmentation(path2,detection_graph, category_index, False)
human_boxes2, human_scores2 = path_util.remove_non_human(boxes2, scores2, classes2)
good_boxes2, good_scores2 = path_util.remove_low_prob(human_boxes2, human_scores2)
good_boxes2, good_scores2 = path_util.remove_poorly_sized_people(good_boxes2, good_scores2)

image3 = Image.open(path3)
width3, height3 = image3.size
image_np3 = path_util.load_image_into_numpy_array(image3)
boxes3, scores3, classes3 = path_util.get_segmentation(path3,detection_graph, category_index, False)
human_boxes3, human_scores3 = path_util.remove_non_human(boxes3, scores3, classes3)
good_boxes3, good_scores3 = path_util.remove_low_prob(human_boxes3, human_scores3)
good_boxes3, good_scores3 = path_util.remove_poorly_sized_people(good_boxes3, good_scores3)

In [3]:
stuff = data_util.collect_data('video_frames/frame', 199, 202, detection_graph, category_index)

TypeError: Required argument 'flags' (pos 10) not found

In [ ]:
person_to_boxes = defaultdict(list)
for entry in stuff[0]:
    if stuff[1][entry['person_id']] != "unknown":
        person_to_boxes[entry['person_id']].append(entry['box'])

In [ ]:
correct = 0
for i in person_to_boxes:
    label = stuff[1][i][0]
    box = person_to_boxes[i][0]
    center_x = (box[3]+box[1])/2.0
    if center_x < 0.5 and label == "l":
        correct+=1
    if center_x >=0.5 and label == "r":
        correct+=1
print(correct)